# С помощью данной методики можно найти свободные IMSI диапазоны, а также создать файл с ключами для отправки кастомеру

In [1]:
# Загрузить данные из файлов
# Задать имена аккаунтов и файлов
#----------------------------------

import pandas as pd
from pandas import DataFrame
from os.path import join, normpath
import datetime as dt
from collections import namedtuple

### Set the variables here ###

reseller_name = 'Ultrapower'
customer_name = 'Ultrapower' # to use as a file name

account_s1 = 'Ultrapower Soft' # Account name with S1
account_s2 = 'Ultrapower Soft' # Account name with S2
account_s4 = '' # Account name with S4
account_s5 = 'Ultrapower Soft' # Account name with S5
account_s6 = 'Ultrapower Soft' # Account name with S6
account_s8 = '' # Account name with S8

##############################

sponsor_object = namedtuple('SponsorRange', 'sponsor test_range prod_range account_name')

s1_sponsor = sponsor_object('S1', '425019613998', '42501', account_s1)
s2_sponsor = sponsor_object('S2', '260060149991', '26006', account_s2)
s4_sponsor = sponsor_object('S4', '454030227305', '45403', account_s4)
s5_sponsor = sponsor_object('S5', '515030191000', '51503', account_s5)
s6_sponsor = sponsor_object('S6', '260036610100', '26003', account_s6)
s8_sponsor = sponsor_object('S8', '234500026400', '23450', account_s8)

downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA\SoftProfiles\Data'
result_file = 'result_file.csv'
files = ['rb_42501_25_06_20181.txt',
         'rb_42501_2_09_20181.txt',
         'rb_26006_25_06_20181.txt',
         'rb_26006_2_09_20181.txt',
         'rb_26006_30_04_2019.txt',
         'rb_42501_30_04_2019.txt',
         'rb_45403_09_07_2019.txt',
         '454030227306000_454030227308999.out',
         '454030227310000_454030227310999.out',
         '515030191000000_515030191099999.out',
         'rb_soft_191103_s1.txt',
         'rb_soft_191103_s2.txt',
         'rb_soft_191103_s4.txt',
         'rb_soft_191107_s6.txt',
         'rb_soft_191107_s8.txt',
         'rb_soft_191207_s1.txt',
         'rb_soft_191207_s2.txt',
         'rb_soft_200317_s4.txt',
         'rb_soft_200317_s8.txt',
         'rb_soft_200723_s6.txt',
         'rb_soft_200917_s1.txt',
         'RB_201009_15K_S1_Soft.txt',
         'RB_201009_15K_S2_Soft.txt',
         'RB_210120_5K_S4_Soft.txt',
         'RB_210120_10K_S6_Soft.txt',]

sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor, s6_sponsor, s8_sponsor]

current_date = dt.datetime.now().strftime('%y%m%d')

df_imsi = DataFrame(columns = ['IMSI', 'KI', 'OPC'])

for file in files:
    df_imsi = df_imsi.append(pd.read_csv(join(downloads, file), sep=' ', engine='python'))

df_imsi.IMSI = df_imsi.IMSI.astype('int64')
df_imsi.sort_values(by = 'IMSI', inplace = True)

df_imsi.head(3)

,IMSI,KI,OPC
0,234500026400000,693E4A067EEB7F66AD9FBD5D0AA3D1E6,79EC10C3C95CED20C758681507904E6D
1,234500026400001,0C12B4359E0AF6FA8D25E39C0662C3D5,02DECFCAE2F9A6D1AED425ED810B2CA6
2,234500026400002,EC3655E2752449C2E8B1F96F79F9A2FD,79798D3604C1EDF288F237963111E7FB


In [8]:
# Из базы данных OCS выгружаем IMSI
#----------------------------------

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT
r.RESELLER_NAME
,a.ACCOUNT_NAME
,ss.STATUS AS SUB_STATUS
,si.IMSI
FROM RESELLERS r
INNER JOIN ACCOUNTS a
ON r.RESELLER_ID = a.RESELLER_ID
INNER JOIN SUBSCRIBERS sr
ON sr.ACCOUNT_ID = a.ACCOUNT_ID
INNER JOIN SUBSCRIBER_STATUS ss
ON ss.SUBSCRIBER_ID = sr.SUBSCRIBER_ID
INNER JOIN SUBSCRIBER_IMSIS si
ON ss.SUBSCRIBER_ID = si.SUBSCRIBER_ID
WHERE
r.deleted is NULL
AND ss.END_DATE IS NULL
AND ss.status = 'Active'
AND
(si.IMSI BETWEEN '260060140890000' AND '260060140899999'
OR si.IMSI BETWEEN '260060149991100' AND '260060149999999'
OR si.IMSI BETWEEN '425019613998060' AND '425019613999999'
OR si.IMSI BETWEEN '425019629920000' AND '425019629939999'
OR si.IMSI BETWEEN '425019613790000' AND '425019613799999'
OR si.IMSI BETWEEN '425019613788000' AND '425019613789999'
OR si.IMSI BETWEEN '260060149982000' AND '260060149989999'
OR si.IMSI BETWEEN '454030227305000' AND '454030227305999'
OR si.IMSI BETWEEN '454030227306000' AND '454030227308999'
OR si.IMSI BETWEEN '454030227310000' AND '454030227310999'
OR si.IMSI BETWEEN '515030191000000' AND '515030191009999'
OR si.IMSI BETWEEN '425019613960000' AND '425019613989999'
OR si.IMSI BETWEEN '260060149940000' AND '260060149969999'
OR si.IMSI BETWEEN '454030227311000' AND '454030227319999'
OR si.IMSI BETWEEN '260036610100000' AND '260036610100999'
OR si.IMSI BETWEEN '234500026400000' AND '234500026400999'
OR si.IMSI BETWEEN '425019613930000' AND '425019613959999'
OR si.IMSI BETWEEN '260060149910000' AND '260060149939999'
OR si.IMSI BETWEEN '454030227615000' AND '454030227619999'
OR si.IMSI BETWEEN '234500026425000' AND '234500026434999'
OR si.IMSI BETWEEN '260036610006000' AND '260036610015999'
OR si.IMSI BETWEEN '425019613096000' AND '425019613105999'
OR si.IMSI BETWEEN '425019613165000' AND '425019613179999'
OR si.IMSI BETWEEN '260060143235000' AND '260060143249999'
OR si.IMSI BETWEEN '454030227645000' AND '454030227649999'
OR si.IMSI BETWEEN '260036610137000' AND '260036610146999')
'''

# OR si.IMSI BETWEEN '515030191000000' AND '515030191099999' # Full range of Smart

df_ocs_imsi = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_ocs_imsi['IMSI'] = df_ocs_imsi['IMSI'].astype('int64')
cnxn.close()
df_ocs_imsi.head(3)

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI
0,1OT,Test SIM cards,Active,425019613998118
1,1OT,Test SIM cards,Active,454030227305015
2,Clementvale,Clementvale S1 profiles,Active,425019613791003


In [9]:
# Объединяем IMSI с ключами и данные OCS
#----------------------------------

df_ocs_imsi_ki = pd.merge(df_imsi, df_ocs_imsi, how='left', on='IMSI')
list_agg = [('IMSI_MIN','min'),('IMSI_MAX','max'),('NUM','count')]
list_col = ['RESELLER_NAME', 'ACCOUNT_NAME', 'SUB_STATUS', df_ocs_imsi_ki.IMSI.astype('str').str.slice(0,12)]
df_ocs_imsi_ki_group = df_ocs_imsi_ki.groupby(list_col)['IMSI'].agg(list_agg).reset_index().sort_values(by='IMSI')

df_ocs_imsi_ki_group.head(3)

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
232,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026400,234500026400054,234500026400999,946
251,Roamability Resellers Test,Showmac Soft,Active,234500026400,234500026400042,234500026400051,10
360,Tottolli-Tech-Mobil,Tottolli S8 Soft,Active,234500026400,234500026400002,234500026400021,20


In [5]:
# For test accounts to use the following ranges:
# S1: 425019613998*
# S2: 260060149991*
# S4: 4540302273050*
# S5: 515030191000*
# Выбрать свободные из "Roamability Invertory"

for sponsor in sponsors:
    print(f'{sponsor.sponsor} test IMSIs in OCS by Reseller:')
    display(df_ocs_imsi_ki_group[df_ocs_imsi_ki_group['IMSI'] == sponsor.test_range].sort_values('IMSI_MIN'))

S1 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
26,Flexiroam Limited -- TESTING at 2018-07,Flexiroam Test Soft simcard,Active,425019613998,425019613998060,425019613998064,5
127,MCN Telecom,MCN Telecom Soft SIM,Active,425019613998,425019613998065,425019613998067,3
13,Dialoq - Genesisclub RT,Genesisclub Partner Soft test,Active,425019613998,425019613998072,425019613998937,18
278,Tottolli-Tech-Mobil,Test Sim cards,Active,425019613998,425019613998079,425019613998081,3
115,Joy_telecom,Joy telecom Partner Soft test,Active,425019613998,425019613998088,425019613998929,5
368,Watchdata SG,Watchdata Partner Soft test,Active,425019613998,425019613998090,425019613998090,1
172,RedTea RT,RedTea TEST Soft-simcards account,Active,425019613998,425019613998093,425019613998491,22
241,Roamability Resellers Test,Clementvale S1 soft,Active,425019613998,425019613998113,425019613998990,6
1,1OT,Test SIM cards,Active,425019613998,425019613998118,425019613998119,2
7,DENT,S1Profiles,Active,425019613998,425019613998120,425019613998142,12


S2 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
25,Flexiroam Limited -- TESTING at 2018-07,Flexiroam Test Soft simcard,Active,260060149991,260060149991100,260060149991104,5
126,MCN Telecom,MCN Telecom Soft SIM,Active,260060149991,260060149991105,260060149991107,3
12,Dialoq - Genesisclub RT,Genesisclub P4 Soft test,Active,260060149991,260060149991110,260060149991945,18
369,Watchdata SG,Watchdata P4 Soft test,Active,260060149991,260060149991123,260060149991123,1
22,Drimsim,Drimsim S2 Soft test IMSI,Active,260060149991,260060149991124,260060149991949,12
0,1OT,Test SIM cards,Active,260060149991,260060149991135,260060149991135,1
8,DENT,S2Profiles,Active,260060149991,260060149991136,260060149991158,12
5,DENT,MultiImsiProfiles,Active,260060149991,260060149991138,260060149991478,21
248,Roamability Resellers Test,PointDume Multi,Active,260060149991,260060149991479,260060149991498,20
146,MCN Telecom,MCN Telecom Soft SIM S2,Active,260060149991,260060149991499,260060149991954,10


S4 test IMSIs in OCS by Reseller:


...

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
24,Drimsim,Drimsim S5 Soft test IMSI,Active,515030191000,515030191000000,515030191000024,25
118,Joy_telecom,Joy telecom S5 Soft,Active,515030191000,515030191000025,515030191000034,10
265,Roamability Test RT,RedTea S5 Test SIM cards,Active,515030191000,515030191000035,515030191000035,1
270,Roamability Test RT,Sigos Test Sim cards,Active,515030191000,515030191000036,515030191000036,1
167,Mondicon SIM RT,S5 soft profiles,Active,515030191000,515030191000037,515030191000056,20
344,Tottolli-Tech-Mobil,Tottolli S5 Soft,Active,515030191000,515030191000057,515030191000756,569
18,Dialoq - Genesisclub RT,Truphone S5 Soft,Active,515030191000,515030191000077,515030191000096,20
255,Roamability Resellers Test,Showmac Soft,Active,515030191000,515030191000097,515030191000106,10
262,Roamability Test RT,DID's,Active,515030191000,515030191000107,515030191000206,100
271,Roamability Test SMART Sponsor RT,Roamability Test SMART,Active,515030191000,515030191000757,515030191000760,4


S6 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
267,Roamability Test RT,Sigos Test Sim cards,Active,260036610100,260036610100000,260036610100000,1
264,Roamability Test RT,Orange Test Sim cards,Active,260036610100,260036610100001,260036610100111,6
359,Tottolli-Tech-Mobil,Tottolli S6 Soft,Active,260036610100,260036610100003,260036610100107,105
257,Roamability Resellers Test,Ultrapower,Active,260036610100,260036610100112,260036610100113,2
221,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610100,260036610100114,260036610100999,886


S8 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
266,Roamability Test RT,Sigos Test Sim cards,Active,234500026400,234500026400000,234500026400000,1
360,Tottolli-Tech-Mobil,Tottolli S8 Soft,Active,234500026400,234500026400002,234500026400021,20
19,Dialoq - Genesisclub RT,Truphone S8 Soft,Active,234500026400,234500026400022,234500026400041,20
251,Roamability Resellers Test,Showmac Soft,Active,234500026400,234500026400042,234500026400051,10
256,Roamability Resellers Test,Ultrapower,Active,234500026400,234500026400052,234500026400053,2
232,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026400,234500026400054,234500026400999,946


In [6]:
# For test accounts to use the following ranges:

for sponsor in sponsors:
    print(f'{sponsor.sponsor} test IMSIs in OCS by Reseller:')
    display(df_ocs_imsi_ki_group[(df_ocs_imsi_ki_group['IMSI'] == sponsor.test_range)
                                & (df_ocs_imsi_ki_group.RESELLER_NAME == 'Roamability Invertory')].sort_values('IMSI_MIN'))

S1 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
193,Roamability Invertory,Partner Soft simcards inventory,Active,425019613998,425019613998538,425019613998900,363


S2 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
214,Roamability Invertory,S4 Soft simcards inventory,Active,260060149991,260060149991555,260060149991908,354


S4 test IMSIs in OCS by Reseller:


...

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
237,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191000,515030191000763,515030191000999,237


S6 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
221,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610100,260036610100114,260036610100999,886


S8 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
232,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026400,234500026400054,234500026400999,946


In [9]:
# For commersial usage

for sponsor in sponsors:
    print(f'{sponsor.sponsor} prod IMSIs in OCS by Reseller:')
    display(df_ocs_imsi_ki_group[(df_ocs_imsi_ki_group.IMSI.str.startswith(sponsor.prod_range, na=False))
                                 & (df_ocs_imsi_ki_group.RESELLER_NAME == 'Roamability Invertory')
                                ].sort_values('IMSI_MIN'))

S1 prod IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
175,Roamability Invertory,Partner Soft simcards inventory,Active,425019613170,425019613170500,425019613170999,500
176,Roamability Invertory,Partner Soft simcards inventory,Active,425019613171,425019613171000,425019613171999,1000
177,Roamability Invertory,Partner Soft simcards inventory,Active,425019613172,425019613172000,425019613172999,1000
178,Roamability Invertory,Partner Soft simcards inventory,Active,425019613173,425019613173000,425019613173999,1000
179,Roamability Invertory,Partner Soft simcards inventory,Active,425019613174,425019613174000,425019613174999,1000
180,Roamability Invertory,Partner Soft simcards inventory,Active,425019613175,425019613175000,425019613175999,1000
181,Roamability Invertory,Partner Soft simcards inventory,Active,425019613176,425019613176000,425019613176999,1000
182,Roamability Invertory,Partner Soft simcards inventory,Active,425019613177,425019613177000,425019613177999,1000
183,Roamability Invertory,Partner Soft simcards inventory,Active,425019613178,425019613178000,425019613178999,1000
184,Roamability Invertory,Partner Soft simcards inventory,Active,425019613179,425019613179000,425019613179999,1000


S2 prod IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
195,Roamability Invertory,S4 Soft simcards inventory,Active,260060143240,260060143240000,260060143240999,1000
196,Roamability Invertory,S4 Soft simcards inventory,Active,260060143241,260060143241000,260060143241999,1000
197,Roamability Invertory,S4 Soft simcards inventory,Active,260060143242,260060143242000,260060143242999,1000
198,Roamability Invertory,S4 Soft simcards inventory,Active,260060143243,260060143243000,260060143243999,1000
199,Roamability Invertory,S4 Soft simcards inventory,Active,260060143244,260060143244000,260060143244999,1000
200,Roamability Invertory,S4 Soft simcards inventory,Active,260060143245,260060143245000,260060143245999,1000
201,Roamability Invertory,S4 Soft simcards inventory,Active,260060143246,260060143246000,260060143246999,1000
202,Roamability Invertory,S4 Soft simcards inventory,Active,260060143247,260060143247000,260060143247999,1000
203,Roamability Invertory,S4 Soft simcards inventory,Active,260060143248,260060143248000,260060143248999,1000
204,Roamability Invertory,S4 Soft simcards inventory,Active,260060143249,260060143249000,260060143249999,1000


S4 prod IMSIs in OCS by Reseller:


...

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
237,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191000,515030191000763,515030191000999,237
238,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191001,515030191001300,515030191001999,700
239,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191008,515030191008000,515030191008999,1000
240,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191009,515030191009000,515030191009999,1000


S6 prod IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
220,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610015,260036610015000,260036610015999,1000
221,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610100,260036610100114,260036610100999,886
222,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610137,260036610137000,260036610137999,1000
223,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610138,260036610138000,260036610138999,1000
224,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610139,260036610139000,260036610139999,1000
225,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610140,260036610140000,260036610140999,1000
226,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610141,260036610141000,260036610141999,1000
227,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610142,260036610142000,260036610142999,1000
228,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610143,260036610143000,260036610143999,1000
229,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610144,260036610144000,260036610144999,1000


S8 prod IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
232,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026400,234500026400054,234500026400999,946
233,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026426,234500026426550,234500026426999,450
234,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026432,234500026432000,234500026432999,1000
235,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026433,234500026433000,234500026433999,1000
236,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026434,234500026434000,234500026434999,1000


In [10]:
# Записать выбранные диапазоны IMSI
# Проверить выбранные IMSI на принадлежность к "Roamability Invertory"

# После переноса IMSI в аккаунты Кастомеров проверить выбранные IMSI и диапазоны
# (предварительно перезагрузив данные из OCS)

imsi_range_object = namedtuple('ImsiRange', 'sponsor imsi_list')

def imsi_list(range_start, range_end):
    return [str(imsi) for imsi in range(range_start, range_end+1,1)]

### Set the IMSI ranges here ###

s1_imsi_range = imsi_range_object('S1', imsi_list(425019613998538, 425019613998538+19))
s2_imsi_range = imsi_range_object('S2', imsi_list(260060149991555, 260060149991555+69))
s4_imsi_range = imsi_range_object('S4', imsi_list(0, 0))
s5_imsi_range = imsi_range_object('S5', imsi_list(515030191000763, 515030191000763+9))
s6_imsi_range = imsi_range_object('S6', imsi_list(260036610100114, 260036610100114+99))
s8_imsi_range = imsi_range_object('S8', imsi_list(0, 0))

#################################

imsi_ranges = [s1_imsi_range, s2_imsi_range, s4_imsi_range, s5_imsi_range, s6_imsi_range, s8_imsi_range]

for imsi_range in imsi_ranges:
    if imsi_range.imsi_list:
        display(df_ocs_imsi[df_ocs_imsi.IMSI.astype('str').isin(imsi_range.imsi_list)].groupby(['RESELLER_NAME',
                                                                                'ACCOUNT_NAME','SUB_STATUS']).count())

,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,
Ultrapower,Ultrapower Soft,Active,20


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,
Ultrapower,Ultrapower Soft,Active,70


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,
Ultrapower,Ultrapower Soft,Active,10


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,
Ultrapower,Ultrapower Soft,Active,100


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,


In [11]:
# Выгружаем файлы с ключами для диапазонов IMSI
#----------------------------------

for sponsor, imsi_range in zip(sponsors, imsi_ranges):
    if sponsor.account_name and imsi_range.imsi_list:
        df_ocs_imsi_ki.loc[(df_ocs_imsi_ki.RESELLER_NAME == reseller_name)
                           & (df_ocs_imsi_ki.ACCOUNT_NAME == sponsor.account_name)
                           & (df_ocs_imsi_ki.IMSI.astype('str').isin(imsi_range.imsi_list)),
                           ['IMSI', 'KI', 'OPC']].\
        to_csv(join(downloads, f'{sponsor.sponsor}_{customer_name}_{current_date}.txt'), index=False)
        print(f'{sponsor.sponsor}_{customer_name}_{current_date}.txt')

S1_Ultrapower_210201.txt
S2_Ultrapower_210201.txt
S5_Ultrapower_210201.txt
S6_Ultrapower_210201.txt


### Далее шифруем и отправляем
----------------------------------
1. Упаковать оба файла в zip архив.
2. Выполнить экспорт открытого ключа Кастомера (дважды кликнуть по файлу ключа).
3. В папке с архивом по пр.кл. на zip архив и выбрать "Зашифровать"
4. Зашифровать своим ключем *_FOR_KEYS
5. Отправить кастомеру. Свой ключ отправлять не нужно.